## Use bonus plates (length of 4)
## Important notes:
### - new color model: number of channels = 4, one channel for each color  
### - plate value encoded with a number: 0.1, 0.4, 1.0  
### - v3.2: all helpers moved to external module
### - v3.2: hint restored: s_before is now field after 'make_move'
### - v3.3: debugging CNN since learning doesn't work
### - v3.4: try bigger CNN 
### - v3.5: problem found. Large space of moves just doesn't work. Reduce to 1

In [10]:
# Needed for tests and a real game against the phone
#import pandas as pd
#import qgrid

import random as rd
import numpy as np
import tensorflow as tf

from collections import deque

import environment as ae

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Constants

In [11]:
# CNN
LEARNING_RATE = 0.001
UPDATE_TARGET_NET = 1000
MINIBATCH_SIZE = 128
GAMMA = 0.99

# Definitions
GAMES_TO_PLAY = 131072
REPLAY_MEMORY_SIZE = 8192
DYNAMIC_LEARNING_EPOCHS = 5
NUMBER_OF_MOVES_IN_GAME = 50

# Variables
MAXIMUM_SCORE = 0
CNN_MOVE_PROB = 0.1
CNN_MOVES_COUNT = 0
CNN_SUCCESSFUL_PREDICTION = 0
AGG_GAMES_NUMBER = 10
TOTAL_SCORE_AGG = 0.0
TOTAL_SUCCESSFUL_MOVES_AGG = 0.0

### Replay Memory Buffer

In [12]:
#
# Replay memory buffer
#
class ExperienceBuffer():
    '''
    Experience Replay Buffer
    Inspired by Andrea Lonza
    '''

    def __init__(self, buffer_size, gamma):
        # Constants
        self.gamma = gamma
        
        # Main Replay Memory buffer parts
        self.states_before = deque(maxlen=buffer_size)
        self.actions = deque(maxlen=buffer_size)
        self.total_rewards = deque(maxlen=buffer_size)
        self.states_after = deque(maxlen=buffer_size)
        self.last_moves = deque(maxlen=buffer_size)
   
    
    def add(self, state_before, action, reward, state_after, last_move):
        # Add certain items to corresponding buffers
        self.states_before.append(state_before)
        self.actions.append(action)
        self.total_rewards.append(reward)
        self.states_after.append(state_after)
        self.last_moves.append(last_move)
    
    
    def sample_minibatch(self, minibatch_size):
        '''
        Sample a minibatch of size batch_size
        Note1: always add the most recent completed move
        '''
        indices = rd.sample(range(len(self.states_before) - 1), minibatch_size - 1)
        # Add the most recent completed move index
        indices.append(len(self.states_before) - 1)
        
        minibatch_states_before = np.array([self.states_before[i] for i in indices]) 
        minibatch_actions = np.array([self.actions[i] for i in indices]) 
        minibatch_total_rewards = np.array([self.total_rewards[i] for i in indices]) 
        minibatch_states_after = np.array([self.states_after[i] for i in indices])  
        minibatch_last_moves = np.array([self.last_moves[i] for i in indices])   
        
        return minibatch_states_before, minibatch_actions, minibatch_total_rewards, minibatch_states_after, minibatch_last_moves
    
    
    def __len__(self):
        '''
        Return length of the current replay memory buffer
        Relevant for the first *minibatch_size* moves.
        '''
        return len(self.states_before)
    
    

## Deep Q-Nework

### Initialize Replay Buffer

In [13]:
replay_memory = ExperienceBuffer(REPLAY_MEMORY_SIZE, GAMMA)

### Initialize Online CNN and Target CNN

In [14]:
tf.random.set_seed(8)

# Initialize optimizer
online_cnn_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)


# Online CNN
Online_CNN = tf.keras.models.Sequential()
Online_CNN.add(tf.keras.layers.Conv2D(256, kernel_size=3, strides = (1, 1), padding='valid', activation=None, data_format='channels_last', kernel_initializer='GlorotUniform', input_shape=(7, 6, 4)))
Online_CNN.add(tf.keras.layers.Activation("relu"))
Online_CNN.add(tf.keras.layers.BatchNormalization())

Online_CNN.add(tf.keras.layers.Flatten())        

Online_CNN.add(tf.keras.layers.Dense(32, activation=None, kernel_initializer='GlorotUniform'))
Online_CNN.add(tf.keras.layers.Activation("relu"))
Online_CNN.add(tf.keras.layers.BatchNormalization())

Online_CNN.add(tf.keras.layers.Dense(1, activation=None, kernel_initializer='GlorotUniform'))
Online_CNN.add(tf.keras.layers.Activation("relu"))


Online_CNN.compile(optimizer=online_cnn_optimizer, loss="mean_squared_error")

# Target CNN
#Target_CNN = tf.keras.models.Sequential()
#Target_CNN.add(tf.keras.layers.Conv2D(128, kernel_size=3, strides = (1, 1), padding='same', activation=tf.keras.activations.tanh, data_format = 'channels_last', kernel_initializer='RandomNormal', input_shape=(7, 6, 4)))
#Target_CNN.add(tf.keras.layers.Conv2D(64, kernel_size=3, strides = (1, 1), padding='same', activation=tf.keras.activations.tanh, kernel_initializer='RandomNormal'))    
#Target_CNN.add(tf.keras.layers.Flatten())                      
#Target_CNN.add(tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, kernel_initializer='RandomNormal'))
#Target_CNN.add(tf.keras.layers.Dense(1, activation=tf.keras.activations.relu, kernel_initializer='RandomNormal'))

#Target_CNN.compile(optimizer=online_cnn_optimizer, loss="mean_squared_error")


# Set weights equal
#Target_CNN.set_weights(Online_CNN.get_weights())

# Just to see how many trainable parameters
Online_CNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 5, 4, 256)         9472      
_________________________________________________________________
activation (Activation)      (None, 5, 4, 256)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 5, 4, 256)         1024      
_________________________________________________________________
flatten_1 (Flatten)          (None, 5120)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                163872    
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 32)               

In [8]:
#def loss(model, X, y_true, A):
#    prediction = model(X)
#    selected_action_values = tf.math.reduce_sum(prediction*A, axis=1)  
#    return tf.keras.losses.MSE(y_true, selected_action_values)


#def grad(model, inputs, targets, actions):
#    with tf.GradientTape() as tape:
#        loss_value = loss(model, inputs, targets, actions)
#    return loss_value, tape.gradient(loss_value, model.trainable_variables)

## Dynamic Learning

In [16]:
%%time

FOUND = False

# Number of moves made to follow the target CNN update strategy
total_moves = 1
                                                      
for game in range(GAMES_TO_PLAY):
    # Start one game
    game_score = 0
    successful_moves = 0

    # Initialize the game field
    field = np.zeros((7, 6, 4))
    field = ae.initialize_field_3D(field)

    for m in range(NUMBER_OF_MOVES_IN_GAME):
        # Total score of one move
        reward = 0

        # Whether CNN made the move
        cnn_made_move_flag = False
        
        # If replay_memory has less than 64 moves, then make a random move
        if ((len(replay_memory) < MINIBATCH_SIZE) or (rd.random() > CNN_MOVE_PROB)):
            move = rd.randint(1, ae.ACTIONS_DIMENSION)
        else:
            # CNN selects a move
            cnn_made_move_flag = True
            CNN_MOVES_COUNT = CNN_MOVES_COUNT + 1
            #_, move = ae.predict_max_score_3D(field.copy(), Target_CNN, ae.ACTIONS_DIMENSION, ae.MOVES)
            _, move = ae.predict_max_score_3D(field.copy(), Online_CNN, ae.ACTIONS_DIMENSION, ae.MOVES)

            
        # Make the move
        new_field, plate_a, plate_b = ae.make_move_v2_3D(field.copy(), move, ae.MOVES)
        field_after_move = new_field.copy()

        # Calculate the score and update the field
        score, new_field = ae.calculate_score_v2_3D(new_field, plate_a, plate_b)
        
        # If the move is successful, then update the field and check if we have new sets        
        successful_move_flag = False

        # While we have new sets (thus the score is greater than 0), process them, calculate score and move plates
        while (score > 0.):
            if (not successful_move_flag):
                successful_moves = successful_moves + 1
                successful_move_flag = True

            # Add new points to the total score of the move
            reward = reward + score

            # Move plates downward, fill the upper row so, that it doesn't have "easy" sets of three
            # Start from the left lower corner (in order to reuse color_fits())
            new_field = ae.fill_field_3D(new_field)

            # Calculate score and check whether we have new sets
            score, new_field = ae.calculate_score_v2_3D(new_field, (-1, -1), (-1, -1))

        # Increase the score of the whole game
        game_score = game_score + reward
        
        # Check whether it's the last move of the current game
        last_move = m == NUMBER_OF_MOVES_IN_GAME - 1         

        # Add new move to the replay memory
        if (successful_move_flag):
            replay_memory.add(field_after_move, move, reward, new_field, last_move)
            
            # Update CNN move statistics
            if (cnn_made_move_flag):    
                CNN_SUCCESSFUL_PREDICTION = CNN_SUCCESSFUL_PREDICTION + 1               
                
        else:
            replay_memory.add(field_after_move, move, 0, field, last_move)   
        
        #
        # Train CNN based on the score
        #
        if (len(replay_memory) >= MINIBATCH_SIZE):
            # Select random MINIBATCH_SIZE moves from replay memory buffer
            samples = replay_memory.sample_minibatch(MINIBATCH_SIZE)

            # Prepare some things for training
            s_before = samples[0]
            actions = samples[1]
            rewards = samples[2]
            s_after = samples[3] 
            dones = samples[4]
            
            # Carefully predict next rewards: we must use predict_max_score
            rewards_next = np.zeros((MINIBATCH_SIZE))
            
            for item in range(MINIBATCH_SIZE):
                # Calculate future rewards ONLY if current reward is NOT zero, otherwise no sense
                if (rewards[item] > 0):
                    #rewards_next[item], _ = ae.predict_max_score_3D(s_after[item], Target_CNN, ae.ACTIONS_DIMENSION, ae.MOVES)
                    rewards_next[item], _ = ae.predict_max_score_3D(s_after[item], Online_CNN, ae.ACTIONS_DIMENSION, ae.MOVES)


            #
            # Update online CNN weights: training step
            #
            actual_values = np.where(dones, rewards, rewards + GAMMA*rewards_next)
            
            #
            # This one doesn't work
            #
            
            #selected_actions = tf.one_hot(actions - 1, ae.ACTIONS_DIMENSION)
            
            #for _ in range(DYNAMIC_LEARNING_EPOCHS):             
            #    loss_value, grads = grad(Online_CNN, s_before, actual_values, selected_actions)
            #    optimizer.apply_gradients(zip(grads, Online_CNN.trainable_variables))
            
            history = Online_CNN.fit(x=s_before, y=actual_values, epochs=DYNAMIC_LEARNING_EPOCHS, verbose=0)
            
            # DEBUG: Trying to find a reason of NaN
            for layer in Online_CNN.layers:
                weights = layer.get_weights()
                for w in weights:
                    if (np.isnan(w).sum() > 0):
                        FOUND = True
                        break
                if (FOUND):
                    break
                    
        if (FOUND):
            print("NaN value found!")
            break

        # If move is successful, update the play field
        if (successful_move_flag):
            field = np.copy(new_field)
            
        # After each UPDATE_TARGET_NET moves update target CNN
        #if (total_moves % UPDATE_TARGET_NET == 0):
        #    Target_CNN.set_weights(Online_CNN.get_weights())
            
        total_moves = total_moves + 1

    if (FOUND):
        break
        
    #
    # Calculate and display overall stats
    #
    # Check whether we have new maximum score
    if (game_score > MAXIMUM_SCORE):
        print(f"New maximum: {game_score}, after {game} games.")
        MAXIMUM_SCORE = game_score
        
    # After each AGG_GAMES_NUMBER games output average game score, average number of successful moves per game
    TOTAL_SCORE_AGG = TOTAL_SCORE_AGG + game_score
    TOTAL_SUCCESSFUL_MOVES_AGG = TOTAL_SUCCESSFUL_MOVES_AGG + successful_moves
    
    if ((game % AGG_GAMES_NUMBER == 0) and (game > 0)):
        avg_score = TOTAL_SCORE_AGG / AGG_GAMES_NUMBER
        TOTAL_SCORE_AGG = 0.0
        
        avg_succ_moves = TOTAL_SUCCESSFUL_MOVES_AGG / AGG_GAMES_NUMBER
        TOTAL_SUCCESSFUL_MOVES_AGG = 0.0

        loss_value = history.history["loss"]
        print(f"Games: {game}, last {AGG_GAMES_NUMBER} games avg score: {avg_score}, avg of succ moves: {avg_succ_moves}, loss: {loss_value}")        
        print(f"CNN made {CNN_MOVES_COUNT} moves. Successful were {CNN_SUCCESSFUL_PREDICTION}")
        print(f"rewards_next is {rewards_next.sum()}")
        
        if (CNN_SUCCESSFUL_PREDICTION / CNN_MOVES_COUNT >= CNN_MOVE_PROB):
            CNN_MOVE_PROB = CNN_MOVE_PROB + 0.1
            
        CNN_MOVES_COUNT = 0
        CNN_SUCCESSFUL_PREDICTION = 0


New maximum: 41, after 0 games.
New maximum: 103, after 1 games.
New maximum: 118, after 10 games.
Games: 10, last 10 games avg score: 74.5, avg of succ moves: 10.4, loss: [889.613525390625, 595.6383666992188, 576.4573364257812, 391.62493896484375, 339.24249267578125]
CNN made 36 moves. Successful were 7
rewards_next is 3262.8039379119873
New maximum: 144, after 14 games.
Games: 20, last 10 games avg score: 78.8, avg of succ moves: 10.8, loss: [173789.4375, 194731.765625, 116686.578125, 169240.953125, 160481.921875]
CNN made 99 moves. Successful were 22
rewards_next is 64588.660217285156
Games: 30, last 10 games avg score: 60.0, avg of succ moves: 9.3, loss: [2103005.0, 1203282.0, 937201.125, 1016950.9375, 836138.125]
CNN made 151 moves. Successful were 30
rewards_next is 112376.56170463562
Games: 40, last 10 games avg score: 67.0, avg of succ moves: 9.3, loss: [23817372.0, 22005104.0, 21802560.0, 20358920.0, 19098000.0]
CNN made 155 moves. Successful were 36
rewards_next is 383792.986

Games: 380, last 10 games avg score: 150.2, avg of succ moves: 20.3, loss: [50229059584.0, 62882058240.0, 45813297152.0, 40326819840.0, 44943400960.0]
CNN made 303 moves. Successful were 166
rewards_next is 32036974.18359375
Games: 390, last 10 games avg score: 187.8, avg of succ moves: 20.3, loss: [150043049984.0, 150773235712.0, 153791528960.0, 144583000064.0, 139119165440.0]
CNN made 295 moves. Successful were 161
rewards_next is 50975666.5
Games: 400, last 10 games avg score: 149.2, avg of succ moves: 18.5, loss: [43524747264.0, 44033540096.0, 40061788160.0, 41150210048.0, 35682344960.0]
CNN made 297 moves. Successful were 149
rewards_next is 29055060.1875
Games: 410, last 10 games avg score: 174.8, avg of succ moves: 20.4, loss: [1021516709888.0, 1020054863872.0, 1025514078208.0, 1000073527296.0, 977609752576.0]
CNN made 301 moves. Successful were 153
rewards_next is 91169810.3125
Games: 420, last 10 games avg score: 173.0, avg of succ moves: 22.3, loss: [35219931136.0, 3212294963

Games: 740, last 10 games avg score: 225.9, avg of succ moves: 29.3, loss: [33158332416000.0, 33282301362176.0, 33216429817856.0, 33030169165824.0, 33081121570816.0]
CNN made 413 moves. Successful were 276
rewards_next is 245307342.375
Games: 750, last 10 games avg score: 204.4, avg of succ moves: 25.1, loss: [336610787328.0, 403249102848.0, 385563131904.0, 483512680448.0, 351459672064.0]
CNN made 405 moves. Successful were 238
rewards_next is 127305910.125
Games: 760, last 10 games avg score: 199.6, avg of succ moves: 23.4, loss: [280298586112.0, 285184262144.0, 225349550080.0, 209562697728.0, 169863266304.0]
CNN made 395 moves. Successful were 213
rewards_next is 132037296.3125
Games: 770, last 10 games avg score: 214.7, avg of succ moves: 26.0, loss: [1348318396416.0, 1307305967616.0, 1263270625280.0, 1334830170112.0, 1228000198656.0]
CNN made 397 moves. Successful were 241
rewards_next is 178390328.28125
Games: 780, last 10 games avg score: 249.4, avg of succ moves: 28.5, loss: [55

Games: 1100, last 10 games avg score: 235.8, avg of succ moves: 29.1, loss: [1005760937984.0, 1027616735232.0, 797872816128.0, 793986465792.0, 664507973632.0]
CNN made 465 moves. Successful were 283
rewards_next is 223756697.33203125
Games: 1110, last 10 games avg score: 252.6, avg of succ moves: 26.9, loss: [1453672890368.0, 1316032217088.0, 1272395071488.0, 1393248436224.0, 1261547552768.0]
CNN made 446 moves. Successful were 256
rewards_next is 254873683.875
Games: 1120, last 10 games avg score: 172.9, avg of succ moves: 22.9, loss: [7976459436032.0, 7890409619456.0, 7736987746304.0, 7776592461824.0, 7532383830016.0]
CNN made 440 moves. Successful were 216
rewards_next is 445067213.5
Games: 1130, last 10 games avg score: 221.7, avg of succ moves: 24.8, loss: [996972691456.0, 961858240512.0, 932073111552.0, 895910608896.0, 743454277632.0]
CNN made 461 moves. Successful were 242
rewards_next is 254251445.125
Games: 1140, last 10 games avg score: 222.5, avg of succ moves: 26.2, loss: [

Games: 1450, last 10 games avg score: 203.5, avg of succ moves: 24.8, loss: [41478411780096.0, 42129069965312.0, 41961536880640.0, 42033288839168.0, 41961306193920.0]
CNN made 453 moves. Successful were 244
rewards_next is 625138704.75
Games: 1460, last 10 games avg score: 167.6, avg of succ moves: 21.7, loss: [12741723029504.0, 9257957195776.0, 13485368934400.0, 8196048551936.0, 8302761082880.0]
CNN made 455 moves. Successful were 208
rewards_next is 326075633.125
Games: 1470, last 10 games avg score: 243.6, avg of succ moves: 27.7, loss: [99699553992704.0, 99456921894912.0, 98808868372480.0, 97366505947136.0, 97555367067648.0]
CNN made 442 moves. Successful were 265
rewards_next is 232762024.625
Games: 1480, last 10 games avg score: 219.1, avg of succ moves: 26.4, loss: [37479985971200.0, 36403937280000.0, 36599656087552.0, 35782316261376.0, 36868540334080.0]
CNN made 447 moves. Successful were 254
rewards_next is 143910769.5
Games: 1490, last 10 games avg score: 192.3, avg of succ m

...

Games: 9630, last 10 games avg score: 1.2, avg of succ moves: 0.4, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 4
rewards_next is 0.0
Games: 9640, last 10 games avg score: 2.8, avg of succ moves: 0.2, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 2
rewards_next is 0.0
Games: 9650, last 10 games avg score: 0.0, avg of succ moves: 0.0, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 0
rewards_next is 0.0
Games: 9660, last 10 games avg score: 0.6, avg of succ moves: 0.2, loss: [0.0703125, 0.0703125, 0.0703125, 0.0703125, 0.0703125]
CNN made 500 moves. Successful were 2
rewards_next is 0.0
Games: 9670, last 10 games avg score: 0.3, avg of succ moves: 0.1, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 1
rewards_next is 0.0
Games: 9680, last 10 games avg score: 1.0, avg of succ moves: 0.1, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 1
rewards_next is 0.0
Games: 9690, last 10 games a

Games: 10130, last 10 games avg score: 1.2, avg of succ moves: 0.3, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 3
rewards_next is 0.0
Games: 10140, last 10 games avg score: 0.0, avg of succ moves: 0.0, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 0
rewards_next is 0.0
Games: 10150, last 10 games avg score: 0.0, avg of succ moves: 0.0, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 0
rewards_next is 0.0
Games: 10160, last 10 games avg score: 0.3, avg of succ moves: 0.1, loss: [0.140625, 0.140625, 0.140625, 0.140625, 0.140625]
CNN made 500 moves. Successful were 1
rewards_next is 0.0
Games: 10170, last 10 games avg score: 0.9, avg of succ moves: 0.3, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 3
rewards_next is 0.0
Games: 10180, last 10 games avg score: 0.0, avg of succ moves: 0.0, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 0
rewards_next is 0.0
Games: 10190, last 10 games

Games: 10640, last 10 games avg score: 0.6, avg of succ moves: 0.2, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 2
rewards_next is 0.0
Games: 10650, last 10 games avg score: 0.6, avg of succ moves: 0.2, loss: [0.0703125, 0.0703125, 0.0703125, 0.0703125, 0.0703125]
CNN made 500 moves. Successful were 2
rewards_next is 0.0
Games: 10660, last 10 games avg score: 0.0, avg of succ moves: 0.0, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 0
rewards_next is 0.0
Games: 10670, last 10 games avg score: 0.3, avg of succ moves: 0.1, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 1
rewards_next is 0.0
Games: 10680, last 10 games avg score: 0.3, avg of succ moves: 0.1, loss: [0.28125, 0.28125, 0.28125, 0.28125, 0.28125]
CNN made 500 moves. Successful were 1
rewards_next is 0.0
Games: 10690, last 10 games avg score: 0.3, avg of succ moves: 0.1, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 1
rewards_next is 0.0
Ga

Games: 11130, last 10 games avg score: 0.3, avg of succ moves: 0.1, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 1
rewards_next is 0.0
Games: 11140, last 10 games avg score: 0.3, avg of succ moves: 0.1, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Successful were 1
rewards_next is 0.0
Games: 11150, last 10 games avg score: 1.8, avg of succ moves: 0.2, loss: [0.421875, 0.421875, 0.421875, 0.421875, 0.421875]
CNN made 500 moves. Successful were 2
rewards_next is 0.0
Games: 11160, last 10 games avg score: 0.6, avg of succ moves: 0.1, loss: [0.0703125, 0.0703125, 0.0703125, 0.0703125, 0.0703125]
CNN made 500 moves. Successful were 1
rewards_next is 0.0
Games: 11170, last 10 games avg score: 0.0, avg of succ moves: 0.0, loss: [0.3515625, 0.3515625, 0.3515625, 0.3515625, 0.3515625]
CNN made 500 moves. Successful were 0
rewards_next is 0.0
Games: 11180, last 10 games avg score: 0.6, avg of succ moves: 0.2, loss: [0.0, 0.0, 0.0, 0.0, 0.0]
CNN made 500 moves. Succe

KeyboardInterrupt: 

In [17]:
print(MAXIMUM_SCORE)

651


## Debuggin the Hint

In [ ]:
# Total score of one move
reward = 0

# Whether CNN made the move
cnn_made_move_flag = False

# If replay_memory has less than 64 moves, then make a random move
if ((len(replay_memory) < MINIBATCH_SIZE) or (rd.random() > CNN_MOVE_PROB)):
    move = rd.randint(1, ae.ACTIONS_DIMENSION)
else:
    # CNN selects a move
    cnn_made_move_flag = True
    CNN_MOVES_COUNT = CNN_MOVES_COUNT + 1
    scr, move = ae.predict_max_score_3D(field.copy(), Target_CNN, ae.ACTIONS_DIMENSION, ae.MOVES)
    print(f"Predicted move {move} with score of {scr}")

In [ ]:
# Make the move
new_field, plate_a, plate_b = ae.make_move_v2_3D(field.copy(), move, ae.MOVES)
field_after_move = new_field.copy()

# Calculate the score and update the field
score, new_field = ae.calculate_score_v2_3D(new_field, plate_a, plate_b)

# If the move is successful, then update the field and check if we have new sets        
successful_move_flag = False

# While we have new sets (thus the score is greater than 0), process them, calculate score and move plates
while (score > 0.):
    if (not successful_move_flag):
        successful_moves = successful_moves + 1
        successful_move_flag = True

    # Add new points to the total score of the move
    reward = reward + score

    # Move plates downward, fill the upper row so, that it doesn't have "easy" sets of three
    # Start from the left lower corner (in order to reuse color_fits())
    new_field = ae.fill_field_3D(new_field)

    # Calculate score and check whether we have new sets
    score, new_field = ae.calculate_score_v2_3D(new_field, (-1, -1), (-1, -1))

print(reward)


In [ ]:

#
# Train CNN based on the score
#

samples = replay_memory.sample_minibatch(MINIBATCH_SIZE)

# Prepare some things for training
s_before = samples[0]
actions = samples[1]
rewards = samples[2]
s_after = samples[3] 
dones = samples[4]


In [ ]:
def loss(model, X, y_true, A):
    prediction = model(X)
    selected_action_values = tf.math.reduce_sum(prediction*A, axis=1)  
    return tf.keras.losses.MSE(y_true, selected_action_values)


def grad(model, inputs, targets, actions):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, actions)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
rewards_next = np.max(Target_CNN(s_after), axis=1)
actual_values = np.where(dones, rewards, rewards + GAMMA*rewards_next)
selected_actions = tf.one_hot(actions, ae.ACTIONS_DIMENSION)

In [ ]:
loss_value, grads = grad(Online_CNN, s_before, actual_values, selected_actions)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(), loss_value.numpy()))

optimizer.apply_gradients(zip(grads, Online_CNN.trainable_variables))

print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(), loss(Online_CNN, s_before, actual_values, selected_actions).numpy()))

### Now the CNN has been trained.
### Start the long reinforcement-learning cycle

In [ ]:
successful_moves

In [ ]:
new_field = make_move(field, move)
print(new_field)

In [ ]:
temp_field = calculate_score(new_field)

In [ ]:
new_field = np.multiply(new_field, 1.0 - temp_field)

In [ ]:
fill_field(new_field, colors)

In [ ]:
#
# Save model
#
# v1: 20190329, trained on len(replay_memory) = 294912
#aero_cnn.save("Aero_CNN_v1")

## Backup

In [ ]:
#
# Create the moves dictionary
#
moves = {}

for i in range(1, 143):
    old_row, old_column, old_direction = process_move_142(i)
    
    start_row = old_row - 1
    start_col = old_column - 1
    
    if (old_direction == "down"):
        end_row = start_row + 1
        end_col = start_col
    elif (old_direction == "up"):
        end_row = start_row - 1
        end_col = start_col
    elif (old_direction == "right"):
        end_row = start_row
        end_col = start_col + 1
    else:
        end_row = start_row
        end_col = start_col - 1
        
    moves[i] = ((start_row, start_col), (end_row, end_col))
    
print(moves)